In [38]:
import sys
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
import nltk
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel

from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt

from gensim.models import KeyedVectors
# NLTK Stop words
from nltk.corpus import stopwords
import gensim.downloader as api
import string
from sklearn.semi_supervised import LabelSpreading, LabelPropagation

# Select your output and input Path

In [39]:
output_path="/home/alip/PycharmProjects/VolunteerismTransfer/data/new/data/"
base_path="/home/alip/PycharmProjects/VolunteerismTransfer/data/"

# Load NLTK English Stop words and  Extend it

In [40]:
stop_words = stopwords.words('english')
stop_words.extend(["http", "https", "rt", "@", ":", "t.co", "co", "amp", "&amp;", "...", "\n", "\r"])
stop_words.extend(string.punctuation)

# Load Input Data(We have 4 Different Input labeled,NGO+ Labaled, Random+Labaled, NGO+Labeled+Random)

In [41]:
fileName="NGO+Random+Labeled Data.json"
df = pd.read_json(output_path+fileName, orient='records', lines=True)

# Preprocess and Tokenize Data 

In [42]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True)
        yield(sent)

In [43]:
data = df.text.values.tolist()
data_words = list(sent_to_words(data))

# Create bigram and trigram and only Keep NOUN,ADJ, VERB and ADV

In [44]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [45]:
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]
    return texts_out

In [46]:
data_ready = process_words(data_words)

# Generate Binary Labels (1 for Volunteer tweets and 0 for non-volunteer Tweets)

In [36]:
def convert_label(label):
    if label == 'Volunteer':
        return 1
    else:
        return 0
df['label'] = [convert_label(x) for x in df['volunteerLabels']]

# Save This Processed Data in seperate Column to use for future use(Either Word Embedding or simple TFIDF)

In [37]:
df['processed_text']=data_ready

df.to_json(output_path+'PR_'+fileName, orient='records', lines=True)

# Load WordEmbedding for First Time

In [20]:
df.shape

(90077, 8)

In [ ]:
import gensim.downloader as api
print(api.load('glove-wiki-gigaword-50', return_path=True))

# Generate word-embedding(Save in sepearet colunm in Data Frame for future use) 

In [31]:
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import roc_auc_score, roc_curve, cohen_kappa_score, precision_score, recall_score, \
    precision_recall_curve

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.semi_supervised import LabelSpreading, LabelPropagation
from tqdm import tqdm

import os
from gensim.models import KeyedVectors
from gensim.downloader import base_dir

# Load Processed Data if necessary

In [ ]:
df = pd.read_json(output_path+'PR_'fileName, orient='records', lines=True)

In [32]:
path = os.path.join(base_dir, 'glove-twitter-100', 'glove-twitter-100.gz')
model_gensim = KeyedVectors.load_word2vec_format(path)

wvs = model_gensim.wv


/home/alip/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [33]:
vectorizer = TfidfVectorizer(
    use_idf=True,
    smooth_idf=False,
    norm=None,  # Applies l2 norm smoothing
    decode_error='replace',
    max_features=10000,
    min_df=4,
    max_df=0.501
)

analyzer = vectorizer.build_analyzer()

In [34]:
def vectorize(sentence):

    tokenized = [t for t in analyzer(sentence)]

    wv_vecs = []
    for t in tokenized:

        try:
            v = wvs[t]
            norm = np.linalg.norm(v)
            normed_v = (v / norm)
            wv_vecs.append(normed_v)
        except:
            continue

    m = np.array(wv_vecs)
    normed_m = np.mean(m, axis=0)

    normed_m = np.nan_to_num(normed_m)


    return normed_m

df['ft_features'] = [vectorize(str(s)) for s in df["processed_text"]]

/home/alip/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/alip/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# Remove tweets with less than 1 words in the text

In [35]:
df['l']=df.apply(lambda row: row['ft_features'].size, axis=1)
ft_data=df.query("l >1")
ft_data.drop( columns=['l'],inplace=True)
ft_data.to_json(output_path+'FT_'+fileName, orient='records', lines=True)

/home/alip/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [36]:
ft_data.shape

(97611, 12)

In [37]:
ft_data.query('label ==1 ').shape

(17512, 12)

In [ ]:
# Load Word Embedding  Data if necessary

In [6]:
ft_data=pd.read_json(output_path+'FT'+fileName, orient='records', lines=True)

# Apply Label Spreading

In [7]:
def lable_propagation(data):
    X_train= data['ft_features']
    src=data['src']
    Y_train=data['label']
    x = []
    y = []

    for index, value in X_train.items():
        x.append(value)
        if src[index]== 'top-accounts' or   src[index]== 'random':
            y.append(-1)
        else:
            y.append(Y_train[index])


    label_prop_model = LabelSpreading(kernel='knn')


    return label_prop_model.fit(x, y).transduction_



In [8]:
ft_data['label']=lable_propagation(ft_data)
ft_data.to_json(output_path+'LS_'+fileName, orient='records', lines=True)